In [1]:
import csv
import openpyxl
from openpyxl import Workbook
import pandas as pd
import pyexcelerate
import line_profiler
import xlwt

In [2]:
print(pd.__version__)
print(openpyxl.__version__)
print(xlwt.__VERSION__)

0.23.3
2.5.3
1.2.0
0.8.0


sampleデータはこちらのページから使わせていただいております。  
http://eforexcel.com/wp/downloads-18-sample-csv-files-data-sets-for-testing-sales/

# csvファイルの読み込み時間だけ計測

## pandas

In [3]:
%%timeit
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")

180 ms ± 62.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 標準ライブラリ csv

In [4]:
%%timeit
with open("50000_Sales_Records.csv", "r") as file:
    data = csv.reader(file)

58.9 µs ± 2.54 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


ただしDataFrameインスタンスを作ることに比べて、csv.readerはその必要がないため必然的に速くなると思える
以下ではcsvを読み込んでから書き込むまでを総合して、時間計測する

# csv読み込みから、書き込みまで時間計測

## pandas

In [5]:
%%timeit
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")
sample_csv_df.to_excel("pandas_output.xlsx", index=False, startrow=1, startcol=1)

16.8 s ± 2.98 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


## openpyxl pattern 1

In [6]:
%%time
with open("50000_Sales_Records.csv", "r") as file:
    data = csv.reader(file)
    wb = Workbook()
    ws = wb.active
    row_num = 1
    for record in data:
        col_num = 1
        for val in record:
            ws.cell(row=row_num, column=col_num, value=val)
            col_num += 1
        row_num += 1
    wb.save("openpyxl_output1.xlsx")

CPU times: user 16 s, sys: 205 ms, total: 16.2 s
Wall time: 16.2 s


## openpyxl pattern 2

In [7]:
%%timeit
with open("50000_Sales_Records.csv", "r") as file:
    data = csv.reader(file)
    wb = Workbook()
    ws = wb.active
    for record in data:
        ws.append(record)  # レコードごとにエクセルに挿入することが可能
    wb.save("openpyxl_output2.xlsx")

14.5 s ± 813 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## xlwt

In [8]:
%%timeit
with open("50000_Sales_Records.csv", "r") as file:
    data = csv.reader(file)
    wb = xlwt.Workbook()
    ws = wb.add_sheet('A Test Sheet')

    row_num = 1
    for record in data:
        col_num = 1
        for val in record:
            ws.write(r=row_num, c=col_num, label=val)
            col_num += 1
        row_num += 1

    wb.save('xlwt_output.xls')  # 注意、拡張子は自由に指定できるが、実際は全てxls形式となっている

7.66 s ± 393 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 結果
| ライブラリ | 計測時間 | 
| :--- | :--- |
| pandas | 15.2 s ± 763 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) | 
| openpyxl 1 | 16.7 s ± 1.29 s per loop (mean ± std. dev. of 7 runs, 1 loop each) | 
| openpyxl 2 | 15.4 s ± 1.15 s per loop (mean ± std. dev. of 7 runs, 1 loop each)| 
| xlwt | 7.62 s ± 510 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) | 


# pandasでengineの指定をxlwtにすると速くなるのか

In [9]:
%%timeit
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")
sample_csv_df.to_excel("pandas_output.xls", index=False, startrow=1, startcol=1, engine="xlwt")

10.9 s ± 462 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## ※速くなるけれどファイル容量が大きくなってしまう

# 最終的な形(後にpyexcelerateに負ける)

In [73]:
%%timeit
output_file_name = "pandas_output.xlsx"
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")
sample_csv_df.to_excel(output_file_name, index=False, startrow=1, startcol=1)

# 色付け、列の幅を整えるためにopenpyxlで読み直す
book = openpyxl.load_workbook(output_file_name)
ws = book.worksheets[0]

fill_pattern = openpyxl.styles.PatternFill(patternType='solid', fgColor='2ecc71')
header = ws[2]  # 2行目を左から順に見ていき、データが最後に存在するカラムまで取得する

min_length = 3
for header_cell in header:
    header_cell.fill = fill_pattern
    
    column = header_cell.column
    column_cells = ws[column]
    length = max(len(str(cell.value)) for cell in column_cells)
    if min_length > length:
        length = min_length
    ws.column_dimensions[column].width = length

book.save(output_file_name)

39.6 s ± 1.6 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


# pyexcelerateを使ってみる

In [11]:
import pyexcelerate
print(pyexcelerate.__version__)

0.8.0


# 以下実験

In [69]:
from pyexcelerate import Workbook, Color, Style
from openpyxl.utils.cell import get_column_letter

In [14]:
%%timeit
with open("50000_Sales_Records.csv", "r") as file:
    data = csv.reader(file)

    wb = Workbook()
    wb.new_sheet("a test sheet", data=data)
    wb.save("pyexcelerate_output.xlsx")

5.02 ms ± 165 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [35]:
%%timeit
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")

wb = Workbook()
wb.new_sheet("a test sheet", data=sample_csv_df.values.tolist())
wb.save("pyexcelerate_output.xlsx")

8.44 s ± 236 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 綺麗にエクセル出力するためには

In [71]:
%%timeit
sample_csv_df = pd.read_csv("50000_Sales_Records.csv")
rownum = len(sample_csv_df.index)
colnum = len(sample_csv_df.columns)

first_cell = "B2"
end_cell = get_column_letter(colnum+1) + str(rownum+1)
header_range = (first_cell, get_column_letter(colnum+1) + str(2))


wb = Workbook()
ws = wb.new_sheet("a test sheet")
ws.range(first_cell, end_cell).value = sample_csv_df.values.tolist()
ws.range(header_range[0], header_range[1]).style.fill.background = Color(46, 204, 113, 0)
for i in range(1, colnum+1):
    ws.set_col_style(i, Style(size=-1))
wb.save("pyexcelerate_output.xlsx")

11.3 s ± 722 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
